In [1]:
import requests
import pandas as pd
import os
from datetime import datetime, timedelta

CITIES = {
    "mumbai": {"lat": 19.0760, "lon": 72.8777}, 
    "delhi": {"lat": 28.6139, "lon": 77.2090},
    "chennai": {"lat": 13.0827, "lon": 80.2707},
    "bangalore": {"lat": 12.9716, "lon": 77.5946}
}

START_DATE = "2022-01-01"
END_DATE = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
AQI_API_URL = "https://air-quality-api.open-meteo.com/v1/air-quality"

/Users/shashanksingh/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/shashanksingh/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
for city_name, coords in CITIES.items():
    
    output_filename = f"{city_name}_raw_pm25.csv"
    if os.path.exists(output_filename):
        print(f"Skipping {city_name} (AQI): File '{output_filename}' already exists.")
        continue

    print(f"\nProcessing AQI for: {city_name.upper()}")
    aqi_params = {
        "latitude": coords["lat"],
        "longitude": coords["lon"],
        "hourly": ["pm2_5"], # Only PM2.5
        "start_date": START_DATE,
        "end_date": END_DATE,
        "timezone": "Asia/Kolkata"
    }
    try:
        response_aqi = requests.get(AQI_API_URL, params=aqi_params)
        response_aqi.raise_for_status() # Raises an error if status is not 200
        
        data_aqi = response_aqi.json()
        aqi_df = pd.DataFrame(data_aqi['hourly'])
        aqi_df.to_csv(output_filename, index=False)
        print(f"Successfully saved: {output_filename}")
        print(f"({city_name} found {aqi_df['pm2_5'].isnull().sum()} null values, which we will clean later)")

    except requests.exceptions.RequestException as e:
        print(f"ERROR: Failed to get AQI data for {city_name}. Skipping. Error: {e}")
        continue

print("\n--- AQI Fetching Complete! ---")

Skipping mumbai (AQI): File 'mumbai_raw_pm25.csv' already exists.
Skipping delhi (AQI): File 'delhi_raw_pm25.csv' already exists.
Skipping chennai (AQI): File 'chennai_raw_pm25.csv' already exists.
Skipping bangalore (AQI): File 'bangalore_raw_pm25.csv' already exists.

--- AQI Fetching Complete! ---
